# cNMF in `cellarium-ml`

Stephen Fleming, Yang Xu

2025.07.31

The `cellarium-ml` project:

https://github.com/cellarium-ai/cellarium-ml

The specific implementation of cNMF we are actively working on:

https://github.com/cellarium-ai/cellarium-ml/pull/196

## Overview

`cellarium-ml` implements a variety of algorithms in a way that is scalable to hundreds of millions of cells and beyond.
This notebook provides a demo run of Cellarium's implementation of consensus NMF (cNMF).
The specific algorithm for NMF is based on "Online learning for matrix factorization and sparse coding" by Mairal, Bach, Ponce, and Sapiro (JMLR 2010).

## This notebook

This notebook shows an end-to-end cNMF run in `cellarium-ml`, starting with h5ad files and ending with results.
There are several steps involved.

## Description of analysis steps

1. Compute highly-variable genes.

2. Run cNMF on selected highly-variable genes.

3. Interactive plotting in this notebook to help determine optimal number of programs `k`, and a 
   `density_threshold` and `local_neighborhood_size` for the consensus step.

4. Computing consensus factors.

5. (Optional) Computing per-cell factor loadings.

6. (Optional) Re-computing the `k` factor definitions using all genes (not just highly-variable genes).

## Inputs

- a lits of filepaths to h5ad files: can be local or in a google bucket (or at some URL)
    - ideally the h5ad files would be from an extracted curriculum from `cellarium-nexus`, but these files can be any h5ad files
        - until [#324](https://github.com/cellarium-ai/cellarium-ml/issues/324) is resolved, the h5ad files
          should be limited in size to what can fit in memory

## Outputs (work in progress... not complete)

- anndata object for all cells (with an empty count matrix) containing:
    - `adata.obsm["X_cnmf_k20"]`: (cell, k) matrix of per-cell factor loadings (for the `k = 20` decomposition)
    - `adata.obsp["cnmf_k20_factors_hvg"]`: (gene, k) matrix of definitions of each of the `k` consensus programs
        - from the initial cNMF fit: all non-highly-variable genes have weight zero
    - `adata.obsp["cnmf_k20_factors_hvg_tpm"]`: (gene, k) matrix of definitions of each of the `k` consensus programs
        - same as above but weights are recomputed to represent TPM values via a refitting step
    - `adata.obsp["cnmf_k20_factors"]`: (gene, k) matrix of definitions of each of the `k` consensus programs
        - computed by refitting cell loadings from `adata.obsp["cnmf_k20_factors_hvg"]` by refitting the dataset including all genes
    - `adata.obsp["cnmf_k20_factors_tpm"]`: (gene, k) matrix of definitions of each of the `k` consensus programs
        - same as above but weights are recomputed to represent TPM values via a refitting step
    - (optionally): all of the above for other choices of `k` as well

NOTE: You will need to use the `nmf_sf_singlenotebook` branch of `cellarium-ml` on github

In [ ]:
import lightning.pytorch as pl
import numpy as np

import cellarium.ml.api
import cellarium.ml.data
import cellarium.ml.models
import cellarium.ml.preprocessing
import cellarium.ml.transforms

In [ ]:
%load_ext autoreload
%autoreload 2

# Data

In [ ]:
# the data

# CZI dataset is PBMCs from https://cellxgene.cziscience.com/e/59b69042-47c2-47fd-ad03-d21beb99818f.cxg/

h5ad_paths = [
    "https://datasets.cellxgene.cziscience.com/a06a1d9e-b1e8-452a-bf83-8ef6ec1044ec.h5ad"
    # "https://storage.googleapis.com/dsp-cellarium-cas-public/test-data/test_0.h5ad",
    # "https://storage.googleapis.com/dsp-cellarium-cas-public/test-data/test_1.h5ad",
    # "https://storage.googleapis.com/dsp-cellarium-cas-public/test-data/test_2.h5ad",
]

In [ ]:
# for demonstration purposes: automatically grab h5ad file paths from a bucket prefix, like data from Nexus

try:
    example_cellarium_curriculum_h5ad_paths = cellarium.ml.api.h5ad_paths_from_google_bucket(
        "gs://cellarium-nexus-file-system-335649/pipeline/data-extracts/TEST2/extract_files"
    )
    print("Example of how to grab files from a Nexus curriculum bucket path:")
    print(f"[{example_cellarium_curriculum_h5ad_paths[0]}, ...]")
except:  # noqa: E722
    pass

## Cellarium data setup

For this demo we are using the python API for cellarium.  It's also possible to use command line versions of these tools.

In [ ]:
h5ad_paths

(For remote files over the internet, this next cell can take a minute.)

In [ ]:
datamodule = cellarium.ml.CellariumAnnDataDataModule(
    dadc=cellarium.ml.data.DistributedAnnDataCollection(
        filenames=h5ad_paths, limits=np.cumsum(cellarium.ml.api.get_h5ad_files_n_cells(h5ad_paths))
    ),
    batch_keys={
        "x_ng": cellarium.ml.utilities.data.AnnDataField(attr="X", convert_fn=cellarium.ml.utilities.data.densify),
        "var_names_g": cellarium.ml.utilities.data.AnnDataField(attr="var_names"),
        "obs_names_n": cellarium.ml.utilities.data.AnnDataField(attr="obs_names"),
    },
    batch_size=1024,
    shuffle=True,
    train_size=1.0,
)

datamodule.setup(stage="fit")

# Highly variable genes

## Run onepass model

This computes mean and variance per gene.

In [ ]:
# get gene names to use later (and assume all files have the same genes)

var_names_g = cellarium.ml.api.get_h5ad_file_var_names_g(h5ad_paths[0])
var_names_g[:3]

In [ ]:
# set up the model that will be used to compute mean and var of each gene

onepass_module = cellarium.ml.CellariumModule(
    transforms=[
        cellarium.ml.transforms.NormalizeTotal(),
        cellarium.ml.transforms.Log1p(),
    ],
    model=cellarium.ml.models.OnePassMeanVarStd(
        var_names_g=cellarium.ml.api.get_h5ad_file_var_names_g(h5ad_paths[0]),
    ),
)

In [ ]:
trainer = pl.Trainer(
    accelerator="cpu",
    devices=1,
    max_epochs=1,
    default_root_dir="tmp/onepass",
)
trainer.fit(onepass_module, datamodule)

In [ ]:
# the onepass model computes a mean and variance per gene

mean_g = trainer.model.model.mean_g
var_g = trainer.model.model.var_g

## Compute hvgs

You can choose `n_top_genes` to suit your needs.

In [ ]:
var = cellarium.ml.preprocessing.get_highly_variable_genes(
    gene_names=var_names_g,
    mean=mean_g,
    var=var_g,
    n_top_genes=2000,
)
var

In [ ]:
var["highly_variable"].sum()

In [ ]:
# the highly variable genes

hvg_var_names_g = var.index[var["highly_variable"]]
hvg_var_names_g

# cNMF

## Run NMF

Set things up to run cNMF in cellarium.

In [ ]:
# user's choice for the number of components: must input a python list

# k_values = [10, 20, 30]
k_values = list(range(3, 20))

In [ ]:
# user's choice for the number of NMF replicates that should go into consensus

nmf_replicates = 100

In [ ]:
# get set up for training

nmf_model = cellarium.ml.models.NonNegativeMatrixFactorization(
    full_g=len(var_names_g),
    var_names_hvg=hvg_var_names_g,
    k_values=k_values,
    r=nmf_replicates,
)

nmf_module = cellarium.ml.CellariumModule(
    cpu_transforms=[cellarium.ml.transforms.Filter(filter_list=hvg_var_names_g)],
    model=nmf_model,
)

datamodule.setup(stage="fit")

trainer_nmf = pl.Trainer(
    accelerator="auto",
    devices=1,
    max_epochs=5,  # this is up for debate, but empirically 5 seems to be enough
    default_root_dir="tmp/nmf",
)

In [ ]:
# train on the data

pl.seed_everything(0)  # not required but helps make this notebook reproducible

trainer_nmf.fit(nmf_module, datamodule)

In [ ]:
# see the shape of the NMF gene programs that have been inferred: [replicates, k, genes]

for k in nmf_model.k_values:
    print(getattr(nmf_model, f"D_{k}_rkg").shape)

## Set up to explore outputs

We have a helper class that facilitates downstream analysis steps. Here we instantiate it and use it to get various outputs.

In [ ]:
from cellarium.ml.models.nmf import NMFOutput

nmf_output = NMFOutput(
    nmf_module=nmf_module,
    datamodule=datamodule,
)

In [ ]:
nmf_output

## Default k-selection plot

This is what Kotliar cNMF would produce with default values for `local_neighborhood_size=0.3` and `density_threshold=0.5`

In [ ]:
nmf_output.default_k_selection_plot()

## Compute consensus factors

This step requires hyperparamter choices.

The way the Kotliar cNMF demo notebook works, you first select a `k` of interest and make these choices for that particular `k`.

Let's start by picking a k-value and looking at what Kotliar calls the "local density histogram".

In [ ]:
best_k = 7

### Density histograms

In [ ]:
nmf_output.plot_density_histograms(k_values=[best_k], local_neighborhood_size=0.3)

In [ ]:
# (optional) re-compute consensus D for certain k-values, given hyperparamter choices

density_threshold = 0.1  # change this threshold if desired based on the histogram above: idea is to remove outliers
local_neighborhood_size = 0.3  # this is a default that Kotliar leaves fixed in tutorials

nmf_output.compute_consensus_factors(
    k_values=[best_k],
    density_threshold=density_threshold,
    local_neighborhood_size=local_neighborhood_size,
)

In [ ]:
# one key for each k-value

nmf_output.consensus.keys()

In [ ]:
# each k-value has a few things computed: these get updated if you re-compute consensus for a particular k value

list(nmf_output.consensus[best_k].keys())

In [ ]:
# the shape of the consensus matrix for a particular k

nmf_output.consensus[best_k]["consensus_D_kg"].shape

### Create clustermap plots

Here we again experiment with different `density_threshold` values. Plotting the clustermap with different `density_threshold` values actually re-computes consensus for that particular `k`.

In [ ]:
nmf_output.plot_clustermap(k=best_k, density_threshold=1.0)

In [ ]:
nmf_output.plot_clustermap(k=best_k, density_threshold=0.1)

## Compute reconstruction error at each k

Demonstration here in case you ever want to run this separately, but it gets run automatically as part of `nmf_output.default_k_selection_plot()`.

Technically the reconstruction error does depend on hyperparamter choices, because it depends on the consensus programs.

In [ ]:
# we need to calculate the reconstruction error by going through the whole dataset: this can take time

nmf_output.calculate_reconstruction_error()

## Other versions of the k-selection plot

Kotliar does not demo this, but the k-selection plot itself depends on the values of `density_threshold` and `local_neighborhood_size`.

In principle, you could choose different hyperparameters for each `k`, run `nmf_output.compute_consensus_factors(k, <your selected params here>)` on all the `k`, and then re-run `nmf_output.calculate_reconstruction_error()` and re-create the k-selection plot.

In [ ]:
nmf_output.k_selection_plot()

See how the stability skyrocketed for `k=best_k` after we filtered at a different threshold?  :)

Different thresholds for different `k` hardly seems fair... unless you were to optimize the threshold for each `k` individually...

### Down a rabbit hole

You know, one guiding principle for this process could be the following... for each `k`, automatically choose a (reasonable) `density_threshold` that maximizes the stability for the given `k`. Let's try it.

In [ ]:
nmf_output.maximal_stability_k_selection_plot()

In [ ]:
# visualize the results of this kind of automatic choosing of density_threshold
# note that the call to plot_clustermap() recomputes consensus, so it's important to specify the computed value

for k in [7, 11]:  # nmf_output.consensus:
    nmf_output.plot_clustermap(k=k, density_threshold=nmf_output.consensus[k]["density_threshold"])

(The strange thing about the stability metric is that it does not guarantee that there are actually `k` clusters.)

## Compute per-cell loadings

In [ ]:
best_k

The loadings of each factor, computed for each cell.

In [ ]:
# get per-cell factor loadings using the best k: this takes time
# `normalize` controls whether the per-cell loadings sum to 1

df = nmf_output.compute_loadings(k=best_k, normalize=True)

In [ ]:
df.shape

In theory could add this information to the anndata object if you had a single object.
Here we will assume the dataset might be very large in total, so we will just try to open one h5ad file and add the annotations for those cells.

In [ ]:
adata = cellarium.ml.data.read_h5ad_file(h5ad_paths[0])
adata.layers["counts"] = adata.raw.X.copy()
adata

In [ ]:
# add cNMF loadings to obsm

adata.obsm["X_nmf"] = df.loc[adata.obs_names].values
adata.obsm["X_nmf"].shape

### Visualize factor loadings on a UMAP

Just for fun, if you have scanpy installed in your environment.

In [ ]:
import scanpy as sc

In [ ]:
sc.set_figure_params(figsize=(5, 5), fontsize=14, vector_friendly=True)

In [ ]:
sc.pp.highly_variable_genes(adata, layer="counts", flavor="seurat_v3", n_top_genes=2000)

In [ ]:
adata.X = adata.layers["counts"].copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
# sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, use_highly_variable=True)
sc.pp.neighbors(adata, method="umap", metric="cosine", n_pcs=10)
sc.tl.umap(adata)
adata.obsm["X_umap_counts"] = adata.obsm["X_umap"].copy()

In [ ]:
# put these in obs for plotting

for k in range(adata.obsm["X_nmf"].shape[1]):
    adata.obs[f"nmf_{k}"] = adata.obsm["X_nmf"][:, k]

In [ ]:
sc.pl.embedding(
    adata, basis="umap_counts", color=["disease_original", "Cell.group", "nmf_0", "nmf_1"], color_map="Oranges", ncols=2
)

In [ ]:
sc.pl.embedding(adata, basis="umap_counts", color=[f"nmf_{i}" for i in range(2, best_k)], color_map="Oranges", ncols=2)

In [ ]:
# try a UMAP derived from the NMF components

sc.pp.neighbors(adata, use_rep="X_nmf", method="umap", metric="cosine")
sc.tl.umap(adata)
adata.obsm["X_umap_nmf"] = adata.obsm["X_umap"].copy()

In [ ]:
sc.pl.embedding(adata, basis="umap_nmf", color=["Cell.group"])

Definitely a bit wonky, but also definitely picking up on cell types.

In [ ]:
sc.pl.embedding(adata, basis="umap", color=["disease_original"])

In [ ]:
# just a random exploration for example

import matplotlib.pyplot as plt
import seaborn as sns

celltype_group = "CD14+ Monocyte"
y = "nmf_1"
sns.boxplot(
    data=adata.obs[adata.obs["Cell.group"] == celltype_group],
    x="disease_original",
    y=y,
    order=["Healthy", "COVID-19 Mild/Remission", "COVID-19 Severe"],
)
plt.gca().set_xticklabels(plt.gca().get_xticklabels(), rotation=60)
plt.title(celltype_group)
plt.ylabel("Per-cell loadings of " + y)
plt.show()

## Explore the factors

Just look at some of the genes involved

In [ ]:
nmf_output.nmf_module.model.var_names_hvg

In [ ]:
import pandas as pd

gene_name_lookup = adata.var["feature_name"].to_dict()

factor_df = pd.DataFrame(
    nmf_output.consensus[best_k]["consensus_D_kg"].t().numpy(), index=nmf_output.nmf_module.model.var_names_hvg
)
factor_df["gene_name"] = factor_df.index.map(gene_name_lookup)
factor_df

In [ ]:
factor_df.sort_values(by=1, ascending=False).head(10)

In [ ]:
factor_df.sort_values(by=2, ascending=False).head(10)

In [ ]:
factor_df.sort_values(by=5, ascending=False).head(10)

In [ ]:
factor_df.sort_values(by=6, ascending=False).head(10)

## Project factors back to all genes

Now refit for all genes, not just the highly variable genes. In cNMF this involves solving an auxiliary linear regression problem.

## Results as a summary anndata

The results can be packaged up into an anndata object if desired, and perhaps saved that way as an h5ad file.

Here we omit the actual count matrix, since in theory it is too big to fit in memory.